# Session 5 — DDL, DML & Transactions (+ Subqueries)

Create/alter tables, modify data safely, and use subqueries for advanced filtering.


## Environment Setup

In [ ]:
import sys, sqlite3, pandas as pd, numpy as np, matplotlib.pyplot as plt
print(sys.version)
import seaborn as sns
sns.set_theme()
from pathlib import Path
DB_PATH = Path('course.db')
conn = sqlite3.connect(DB_PATH)
conn.execute('PRAGMA foreign_keys=ON;')
print('SQLite ready at', DB_PATH.resolve())

In [ ]:
def run_sql(q, params=None):
    params = params or {}
    df = pd.read_sql_query(q, conn, params=params)
    display(df)
    return df

## 1. DDL — Alter Table

In [ ]:
conn.execute("ALTER TABLE customers ADD COLUMN loyalty_tier TEXT DEFAULT 'Standard';");
conn.commit()
run_sql("SELECT * FROM customers;")

## 2. DML and Transactions

In [ ]:
print("Before:")
run_sql("SELECT order_id, SUM(quantity) AS total_qty FROM order_items GROUP BY order_id ORDER BY order_id;")
try:
    conn.execute("BEGIN;")
    conn.execute("UPDATE order_items SET quantity = quantity + 1 WHERE order_id IN (1,2);")
    print("During (uncommitted):")
    run_sql("SELECT order_id, SUM(quantity) AS total_qty FROM order_items GROUP BY order_id ORDER BY order_id;")
    conn.execute("ROLLBACK;")
finally:
    pass
print("After rollback:")
run_sql("SELECT order_id, SUM(quantity) AS total_qty FROM order_items GROUP BY order_id ORDER BY order_id;")

## 3. Subqueries Example

In [ ]:
run_sql('''
SELECT o.order_id, COUNT(*) AS n_items
FROM order_items oi
JOIN orders o ON oi.order_id=o.order_id
GROUP BY o.order_id
HAVING n_items > (SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt FROM order_items GROUP BY order_id) t);
''')